In [11]:
from dotenv import load_dotenv
load_dotenv()
cohere_api_key=os.getenv("COHERE_API_KEY")
alpha_vantage_key=os.getenv("ALPHA_VANTAGE_API_KEY")

In [28]:
import os
import requests
import pandas as pd
import numpy as np
from typing import Dict, Any

from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatCohere
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field

class FinancialDataFetcher(BaseTool):
    name: str = "financial_data_fetcher"  # Add type annotation
    description: str = """
    Fetches financial data for a given stock ticker using Alpha Vantage API.
    Retrieves key financial metrics including:
    - Income Statement
    - Balance Sheet
    - Cash Flow Statement
    """

    class InputSchema(BaseModel):
        ticker: str = Field(..., description="Stock ticker symbol")
        api_key: str = Field(..., description="Alpha Vantage API Key")

    def _run(self, ticker: str, api_key: str):
        """
        Fetch financial statements for a given stock ticker
        """
        try:
            # Income Statement Endpoint
            income_url = f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={api_key}"
            income_response = requests.get(income_url).json()
            
            # Balance Sheet Endpoint
            balance_url = f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={api_key}"
            balance_response = requests.get(balance_url).json()
            
            # Validate and extract most recent annual report
            if 'annualReports' not in income_response or not income_response['annualReports']:
                return "No annual financial data available for the given ticker."
            
            # Extract key financial metrics from the most recent annual report
            latest_report = income_response['annualReports'][0]
            latest_balance_sheet = balance_response['annualReports'][0]
            
            financial_metrics = {
                "Ticker": ticker,
                "Net Income": float(latest_report.get('netIncome', 0)),
                "Total Revenue": float(latest_report.get('totalRevenue', 0)),
                "Gross Profit": float(latest_report.get('grossProfit', 0)),
                "Total Assets": float(latest_balance_sheet.get('totalAssets', 0)),
                "Total Liabilities": float(latest_balance_sheet.get('totalLiabilities', 0)),
                "Shareholders Equity": float(latest_balance_sheet.get('totalShareholderEquity', 0))
            }
            
            return financial_metrics
        
        except Exception as e:
            return f"Error fetching financial data: {str(e)}"

    def _arun(self, **kwargs):
        raise NotImplementedError("Async not supported for this tool")


class FinancialRatiosCalculator(BaseTool):
    name: str = "financial_ratios_calculator"  # Add type annotation
    description: str = """
    Calculates key financial ratios using fetched financial data.
    Supports calculations for:
    - Return on Equity (ROE)
    - Return on Assets (ROA)
    - Gross Margin
    - Net Profit Margin
    - Debt to Equity Ratio
    """

    class InputSchema(BaseModel):
        financial_metrics: Dict[str, float] = Field(..., description="Financial metrics dictionary")

    def _run(self, financial_metrics: Dict[str, float]):
        try:
            # Calculate Financial Ratios
            roe = (financial_metrics['Net Income'] / financial_metrics['Shareholders Equity']) * 100 if financial_metrics['Shareholders Equity'] != 0 else 0
            roa = (financial_metrics['Net Income'] / financial_metrics['Total Assets']) * 100 if financial_metrics['Total Assets'] != 0 else 0
            gross_margin = (financial_metrics['Gross Profit'] / financial_metrics['Total Revenue']) * 100 if financial_metrics['Total Revenue'] != 0 else 0
            net_profit_margin = (financial_metrics['Net Income'] / financial_metrics['Total Revenue']) * 100 if financial_metrics['Total Revenue'] != 0 else 0
            debt_to_equity = (financial_metrics['Total Liabilities'] / financial_metrics['Shareholders Equity']) if financial_metrics['Shareholders Equity'] != 0 else 0

            return {
                "Ticker": financial_metrics['Ticker'],
                "Return on Equity (ROE)": f"{roe:.2f}%",
                "Return on Assets (ROA)": f"{roa:.2f}%", 
                "Gross Margin": f"{gross_margin:.2f}%",
                "Net Profit Margin": f"{net_profit_margin:.2f}%",
                "Debt to Equity Ratio": f"{debt_to_equity:.2f}"
            }
        except Exception as e:
            return f"Error calculating financial ratios: {str(e)}"

    def _arun(self, **kwargs):
        raise NotImplementedError("Async not supported for this tool")

def create_financial_analysis_agent(cohere_api_key: str, alpha_vantage_api_key: str):
    """
    Create a LangChain agent for financial data fetching and ratio analysis
    
    Args:
        cohere_api_key (str): API key for Cohere LLM
        alpha_vantage_api_key (str): API key for Alpha Vantage
    
    Returns:
        Initialized LangChain agent for financial analysis
    """
    # Initialize Cohere LLM
    llm = ChatCohere(cohere_api_key=cohere_api_key, model="command")
    
    # Create financial tools
    financial_tools = [
        FinancialDataFetcher(),
        FinancialRatiosCalculator()
    ]
    
    # Initialize agent
    agent = initialize_agent(
        tools=financial_tools, 
        llm=llm, 
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    
    return agent

def main():
    
    # Create financial analysis agent
    financial_agent = create_financial_analysis_agent(cohere_api_key, alpha_vantage_key)
    
    # Stock ticker to analyze (e.g., AAPL for Apple, MSFT for Microsoft)
    stock_ticker = "AAPL"
    
    # Two-step process:
    # 1. Fetch financial data
    financial_data_tool = FinancialDataFetcher()
    financial_metrics = financial_data_tool._run(stock_ticker, alpha_vantage_key)
    
    # 2. Calculate financial ratios
    ratios_tool = FinancialRatiosCalculator()
    financial_analysis = ratios_tool._run(financial_metrics)
    
    print(f"Financial Analysis for {stock_ticker}:")
    for key, value in financial_analysis.items():
        print(f"{key}: {value}")

In [29]:
main()

Financial Analysis for AAPL:
Ticker: AAPL
Return on Equity (ROE): 164.59%
Return on Assets (ROA): 25.68%
Gross Margin: 46.21%
Net Profit Margin: 23.97%
Debt to Equity Ratio: 5.41
